In [1]:
import pandas as pd
import numpy as np

: 

In [ ]:
dat = pd.read_csv('../raw_data/bgg_1000_data.csv')

dat.columns= ['idx', 'name', 'published_year', 'avg_rating', 'recc_plyr', 'play_time', 'recc_age', 'weight', 'game_type',
             'mech_1', 'mech_2', 'mech_3', 'mech_4', 'mech_5']

rank = range(1,1001)
dat['rank'] = rank

In [144]:
#함수?

def dat_handling(data) :
    
    # 발매 연도 열의 필요없는 소괄호 제거 및 정수화 진행
    data['published_year'] = data['published_year'].str.replace(r'\(|\)', '', regex=True).astype(int)
    
    '''
    추천 플레이어 열 만들기
    각 문자열을 나누어 1col은 min player, 3열은 max player로 나누어 저장한다.
    결측치(2인용)의 경우엔 max_player에 None이 저장된다.
    '''
    
    ply_colname = ['min_player', 'max_player']
    split_result = data['recc_plyr'].str.split(r'[– ]', expand=True) # 문자열 공백 및 - 기준으로 분리 후 각각의 열로 저장
    split_result = split_result.iloc[:, [0,1]] # 그 중 필요한 숫자가 있는 열만 가져온다.

    split_result.loc[:,1] = split_result.apply(
    lambda row: row[0] if row[1] == "Players" else row[1], axis = 1 #각 row[1] 마다 Players이면 이전row인 row[0]로 바꿈
        )
    split_result.columns = ply_colname
    res_data = pd.concat([data, split_result], axis = 1)
    
    '''
    추천 플레이 시간 열 만들기
    플레이 시간의 숫자 처리도 동일하게 적용한다.
    '''

    ply2_colname = ['min_playtime', 'max_playtime']
    split_result2 = data['play_time'].str.split(r'[– ]', expand=True) # 문자열 ' ', '–' 기준으로 분리 후 각각의 열로 저장
    split_result2 = split_result2.iloc[:, [0,1]] # 분리 결과, 추천 시간대가 한 개인 경우는 최대 시간대가 Min 이 된다.

    #그놈의 체스 때문에....
    split_result2.iloc[:,0] = split_result2.apply(
        lambda row: 60 if row[0] == "Time:" else row[0], axis = 1 
    )
    
    #각 row[1] 마다 Players이면 이전row인 row[0]로 바꿈
    split_result2.iloc[:,1] = split_result2.apply(
        lambda row: row[0] if row[1] == "" or row[1] == "Min" else row[1], axis = 1 
    )
    split_result2.columns = ply2_colname
    res_data = pd.concat([res_data, split_result2], axis = 1)

    '''
    recc_age 처리
    : 와 +로 문자열을 나누어서 중간에 있는 숫자(1열)만 빼오도록 한다.
     '--' 로 추천 나이가 지정되지 않은 행이 존재한다 -> 어떻게 처리?
    '''

    res_data['recc_age'] = res_data['recc_age'].str.split(r'[:+]', expand = True)[1]
    res_data['recc_age'] = res_data['recc_age'].replace(' ––', 14).astype(int)

    '''
    game_type 처리
    , 와 \으로 나누어 0번째 요소만 주요 게임 타입으로 설정한다.
    '''

    res_data['game_type'] = res_data['game_type'].str.split(r'[\n\,]', expand = True)[0]

    '''
    #필요없는 열 제거
    반복문으로 더 깔끔하게 만들기
    '''
    del_cols = ['recc_plyr', 'play_time','idx']
    for col in del_cols:
        res_data.drop(col, axis = 1, inplace = True)

    return res_data #완성!

In [ ]:
'''
data = dat.copy()

ply2_colname = ['min_playtime', 'max_playtime']
split_result2 = data['play_time'].str.split(r'[– ]', expand=True) # 문자열 ' ', '–' 기준으로 분리 후 각각의 열로 저장
split_result2 = split_result2.iloc[:, [0,1]] # 분리 결과, 추천 시간대가 한 개인 경우는 최대 시간대가 Min 이 된다.

split_result2.iloc[:,0] = split_result2.apply(
        lambda row: 60 if row[0] == "Time:" else row[0], axis = 1 #각 row[1] 마다 Players이면 이전row인 row[0]로 바꿈
    )
split_result2.iloc[:,1] = split_result2.apply(
        lambda row: row[0] if row[1] == "" or row[1] == "Min" else row[1], axis = 1 #각 row[1] 마다 Players이면 이전row인 row[0]로 바꿈
    )

split_result2.columns = ply2_colname #칼럼 명 지정

print(split_result2.iloc[441])

#min_playtime_distribution = split_result2['min_playtime'].value_counts()
#print(min_playtime_distribution)
'''

In [ ]:
'''
# 추천 나이대 str값 오류 해결

data = dat.copy()

data['recc_age'] = data['recc_age'].str.split(r'[:+]', expand = True)[1]
if data[data['recc_age'] == ' ––'] :
    data['recc_age'] == "14"

print(data)
'''

In [ ]:
df = dat_handling(dat)
df.info()

In [147]:
df.to_csv('../raw_data/BGG_data.csv', index = False)

In [ ]:
df.iloc[441]